<a href="https://colab.research.google.com/github/Akshaykumarmundrathi/Oklahoma-Well-Locations/blob/main/oklahoma_well_location_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# ========================================
# 🔹 Oklahoma Well Locations Interactive Map
# ========================================

# Step 1: Install dependencies
!pip install pandas folium openpyxl

# Step 2: Import libraries
import pandas as pd
import folium


In [13]:
# Colab-ready: Extract TRS + randymajors coords + OCR fallback + optional 8x8 grid refinement
# Run in Google Colab.
!pip install PyMuPDF pytesseract pdf2image pandas openpyxl requests pyproj opencv-python-headless tqdm

import os, io, re, math, tempfile
from tqdm import tqdm
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import requests
import pandas as pd
from google.colab import files
import numpy as np
import cv2
from pyproj import Transformer
!apt-get -y install poppler-utils


ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd


In [ ]:

pdf_path = "PDFs_Well_Location_20250523_140405.pdf"

def extract_lat_lon_from_link_text(link_text):
    """
    Extract latitude and longitude from a randymajors.org URL string,
    accounting for a semicolon immediately after y parameter (y;=val).
    Returns (latitude, longitude) as floats or (None, None) if not found.
    """
    # Regex to match x=... and y=... where y may be followed by a semicolon
    pattern = re.compile(
        r"x=([-+]?\d*\.\d+|\d+)[;&]?\s*y;?=([-+]?\d*\.\d+|\d+)",
        re.IGNORECASE
    )
    # Remove newlines which sometimes disrupt the URL in PDFs
    link_text = link_text.replace('\n', '')
    match = pattern.search(link_text)
    if match:
        try:
            lon = float(match.group(1))
            lat = float(match.group(2))
            return lat, lon
        except ValueError:
            pass
    return None, None


doc = fitz.open(pdf_path)
all_text = "\n".join(page.get_text("text") for page in doc)

# Split into report blocks separated by 'Report for:'
raw_blocks = re.split(r"Report for:\s*", all_text, flags=re.IGNORECASE)
blocks = [b.strip() for b in raw_blocks if b.strip()]

records = []

for block in blocks:
    lines = block.split("\n")
    # Filter empty lines and strip spaces
    lines = [line.strip() for line in lines if line.strip()]
    report_name = lines[0] if lines else None
    county = section = township = rng = link = lat = lon = None

    for i, line in enumerate(lines):
        if re.match(r"County:", line):
            county_match = re.search(r"County:\s*([^,]+)", line)
            section_match = re.search(r"Section:\s*([0-9]{1,3})", line)
            township_match = re.search(r"Township:\s*([\d]+[ ]?[NS])", line)
            range_match = re.search(r"Range:\s*([\d]+[ ]?[EW])", line)

            county = county_match.group(1).strip() if county_match else None
            section = section_match.group(1).strip() if section_match else None
            township = township_match.group(1).strip() if township_match else None
            rng = range_match.group(1).strip() if range_match else None

            # Check next line for the link
            if i + 1 < len(lines):
                possible_link = lines[i + 1]
                if possible_link.startswith("http"):
                    link = possible_link
                    lat, lon = extract_lat_lon_from_link_text(possible_link)
            break  # Consider only first County line per block

    records.append({
        "Report Name": report_name,
        "County": county,
        "Section": section,
        "Township": township,
        "Range": rng,
        "Link": link,
        "Latitude": lat,
        "Longitude": lon
    })

df = pd.DataFrame(records)
out_file = "oklahoma_wells_text_extract_with_latlon.csv"
df.to_csv(out_file, index=False)

print(f"Extracted {len(df)} reports, {df[['Latitude','Longitude']].dropna().shape[0]} have coordinates.")
print(df.head(10))


In [ ]:
# ========================================
# 🔹 Oklahoma Well Locations Interactive Map
# ========================================

# Step 1: Install dependencies (run once)
!pip install pandas folium openpyxl

# Step 2: Import libraries
import pandas as pd
import folium


In [ ]:
# Step 1: Install dependencies (run if needed)
!pip install pandas folium openpyxl

# Step 2: Imports
import pandas as pd
import folium

# Step 3: Load your CSV data
df = pd.read_csv("/content/oklahoma_wells_text_extract_with_latlon.csv")

# Step 4: Create map centered on Oklahoma
oklahoma_center = [35.4676, -97.5164]
m = folium.Map(location=oklahoma_center, zoom_start=7, tiles="OpenStreetMap")

# Step 5: Add well markers for rows with valid lat/lon
for _, row in df.iterrows():
    if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=(
                f"<b>{row['Report Name']}</b><br>"
                f"County: {row['County']}<br>"
                f"Section: {row['Section']}<br>"
                f"Township: {row['Township']}<br>"
                f"Range: {row['Range']}<br>"
                f"Lat: {row['Latitude']}<br>"
                f"Lon: {row['Longitude']}"
            ),
            icon=folium.Icon(color="red", icon="tint", prefix="fa"),
        ).add_to(m)

# Step 6: Display map in notebook (Jupyter/Colab)
m

# Step 7: Save to HTML (optional)
m.save("oklahoma_wells_map.html")
print("✅ Map saved as 'oklahoma_wells_map.html'.")


In [14]:
# ===============================================
# 🔹 Oklahoma Well Locations Extraction + Map
# ===============================================
# Original logic preserved — structured, documented, and debug-safe.

# ------------------------------------------------
# STEP 0 — Install dependencies (run once)
# ------------------------------------------------
# pip install PyMuPDF pytesseract pdf2image pandas openpyxl requests pyproj opencv-python-headless tqdm folium
# apt-get -y install poppler-utils  # Linux/Colab only

# ------------------------------------------------
# STEP 1 — Imports
# ------------------------------------------------
import os, re, fitz, pandas as pd
from tqdm import tqdm
import folium

# ------------------------------------------------
# STEP 2 — Input PDF and output filenames
# ------------------------------------------------
pdf_path = "PDFs_Well_Location_20250523_140405.pdf"     # your input PDF
out_csv  = "oklahoma_wells_text_extract_with_latlon.csv"
out_html = "oklahoma_wells_map.html"

# ------------------------------------------------
# STEP 3 — Helper: extract lat/lon from randymajors link
# ------------------------------------------------
def extract_lat_lon_from_link_text(link_text):
    """
    Extract latitude/longitude from a randymajors.org URL string.
    Handles 'y;=' or normal 'y=' syntax.
    Input:  full link string
    Output: (lat, lon) floats or (None, None)
    """
    pattern = re.compile(
        r"x=([-+]?\d*\.\d+|\d+)[;&]?\s*y;?=([-+]?\d*\.\d+|\d+)",
        re.IGNORECASE
    )
    link_text = link_text.replace('\n', '')
    m = pattern.search(link_text)
    if not m:
        return None, None
    try:
        lon = float(m.group(1))
        lat = float(m.group(2))
        return lat, lon
    except Exception:
        return None, None

# ------------------------------------------------
# STEP 4 — Read entire PDF text
# ------------------------------------------------
try:
    doc = fitz.open(pdf_path)
except Exception as e:
    raise SystemExit(f"❌ Cannot open PDF: {pdf_path}\n{e}")

all_text = "\n".join(page.get_text("text") for page in doc)
doc.close()

# ------------------------------------------------
# STEP 5 — Split into individual report blocks
# ------------------------------------------------
raw_blocks = re.split(r"Report for:\s*", all_text, flags=re.IGNORECASE)
blocks = [b.strip() for b in raw_blocks if b.strip()]

# ------------------------------------------------
# STEP 6 — Parse each report for metadata and coords
# ------------------------------------------------
records = []
for block in tqdm(blocks, desc="Processing reports"):
    lines = [ln.strip() for ln in block.split("\n") if ln.strip()]
    report_name = lines[0] if lines else None
    county = section = township = rng = link = lat = lon = None

    for i, line in enumerate(lines):
        if re.match(r"County:", line):
            county_match   = re.search(r"County:\s*([^,]+)", line)
            section_match  = re.search(r"Section:\s*([0-9]{1,3})", line)
            township_match = re.search(r"Township:\s*([\d]+[ ]?[NS])", line)
            range_match    = re.search(r"Range:\s*([\d]+[ ]?[EW])", line)

            county   = county_match.group(1).strip()   if county_match   else None
            section  = section_match.group(1).strip()  if section_match  else None
            township = township_match.group(1).strip() if township_match else None
            rng      = range_match.group(1).strip()    if range_match    else None

            # Possible link on the next line
            if i + 1 < len(lines):
                next_line = lines[i + 1]
                if next_line.startswith("http"):
                    link = next_line
                    lat, lon = extract_lat_lon_from_link_text(next_line)
            break

    records.append({
        "Report Name": report_name,
        "County": county,
        "Section": section,
        "Township": township,
        "Range": rng,
        "Link": link,
        "Latitude": lat,
        "Longitude": lon
    })

# ------------------------------------------------
# STEP 7 — Save extracted data
# ------------------------------------------------
df = pd.DataFrame(records)
df.to_csv(out_csv, index=False)
print(f"✅ Extracted {len(df)} reports. "
      f"{df[['Latitude','Longitude']].dropna().shape[0]} have coordinates.")
print(df.head())

# ------------------------------------------------
# STEP 8 — Create interactive Folium map
# ------------------------------------------------
oklahoma_center = [35.4676, -97.5164]
m = folium.Map(location=oklahoma_center, zoom_start=7, tiles="OpenStreetMap")

for _, row in df.iterrows():
    if pd.notna(row["Latitude"]) and pd.notna(row["Longitude"]):
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=(
                f"<b>{row['Report Name']}</b><br>"
                f"County: {row['County']}<br>"
                f"Section: {row['Section']}<br>"
                f"Township: {row['Township']}<br>"
                f"Range: {row['Range']}<br>"
                f"Lat: {row['Latitude']}<br>"
                f"Lon: {row['Longitude']}"
            ),
            icon=folium.Icon(color="red", icon="tint", prefix="fa")
        ).add_to(m)

m.save(out_html)
print(f"✅ Map saved as '{out_html}'.  Open this file in a browser to view markers.")


Processing reports: 100%|██████████| 3002/3002 [00:00<00:00, 48710.04it/s]


✅ Extracted 3002 reports. 1155 have coordinates.
                Report Name           County Section Township Range  \
0  00000000_BUNCH 1_1400935             None    None     None  None   
1  00000000_No Data_1030447  Okmulgee County      11     14 N  12 E   
2  00000000_No Data_1220780             None    None     None  None   
3  00000000_No Data_1220867     Tulsa County      34     19 N  12 E   
4  00000000_No Data_1220868  Okmulgee County      21     15 N  12 E   

                                                Link   Latitude  Longitude  
0                                               None        NaN        NaN  
1  https://www.randymajors.org/township-range-on-...  35.701756 -96.004664  
2                                               None        NaN        NaN  
3  https://www.randymajors.org/township-range-on-...  36.076980 -96.026723  
4  https://www.randymajors.org/township-range-on-...  35.767142 -96.050405  
✅ Map saved as 'oklahoma_wells_map.html'.  Open this file in a